In [161]:
import kfp
import kfp.components as comp
import kfp.dsl as dsl
from kfp.v2.dsl import (
    component,
    Output,
    ClassificationMetrics,
    Metrics,
    HTML,
    Markdown
)

In [162]:
def exploring_dataset(sizetreina:float) -> float:
    from sklearn.datasets import load_iris # Load Iris Data
    import pandas as pd
    import numpy as np
    
    iris = load_iris() # Creating pd DataFrames
    #print("Conteudo do Load Iris")
    #print(iris)
    
    
    iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
    
    #print("Conteudo do Iris DF")
    #print(iris_df)
    
    
    target_df = pd.DataFrame(data= iris.target, columns= ['species'])
    #print("-------------------------------------")    
    #print("Conteudo do Iris DF")
    #print(target_df)
    
    species = []

    for i in range(len(target_df['species'])):
        if target_df['species'][i] == 0:
            species.append("setosa")
        elif target_df['species'][i] == 1:
            species.append('versicolor')
        else:
            species.append('virginica')
            
    #print("-------------------------------------")    
    #print("Conteudo do Array species")
    #print(species)
            

    target_df['species'] = species
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe target_df")
    #print(target_df)
        
    iris_df = pd.concat([iris_df, target_df], axis= 1)
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe iris_df")
    #print(iris_df)
    
    
    from sklearn.model_selection import train_test_split  
    from sklearn.linear_model import LogisticRegression
    from sklearn.linear_model import LinearRegression
    
    
    # Converting Objects to Numerical dtype
    iris_df.drop('species', axis= 1, inplace= True)
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe iris_df pos Drop")
    #print(iris_df)
    
    target_df = pd.DataFrame(columns= ['species'], data= iris.target)
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe target_df 2")
    #print(target_df)
    
    
    iris_df = pd.concat([iris_df, target_df], axis= 1)# Variables
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe IRIS Final")
    #print(iris_df)    
    
    
    X= iris_df.drop(labels= 'sepal length (cm)', axis= 1)
    y= iris_df['sepal length (cm)']

    # Splitting the Dataset 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= sizetreina, random_state= 101)
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe X_train")
    #print(X_train)  
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe X_test")
    #print(X_test)  
    
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe y_train")
    #print(y_train)  
    #print("-------------------------------------")    
    #print("Conteudo do Dataframe y_test")
    #print(y_test)  

    

    # Instantiating LinearRegression() Model
    lr = LinearRegression()

    # Training/Fitting the Model
    lr.fit(X_train, y_train)
    
    # Making Predictions
    lr.predict(X_test)
    
    
    #print("-------------------------------------")    
    #print("Conteudo do lr.predict(X_test)")
    #print(lr.predict(X_test))
    #print("-------------------------------------")        
    
    
    pred = lr.predict(X_test)

    from sklearn import metrics
    from sklearn.metrics import mean_absolute_error
    from sklearn.metrics import mean_squared_error

    #print("-------------------------------------")   
    #print("Reports Prediction")
    #print(metrics.classification_report(y_test, pred, digits=3))        
        
        
    # Evaluating Model's Performance
    print('Mean Absolute Error:', mean_absolute_error(y_test, pred))
    #print('Mean Squared Error:', mean_squared_error(y_test, pred))
    #print('Mean Root Squared Error:', np.sqrt(mean_squared_error(y_test, pred))) 
    #print('r2_score ', metrics.r2_score(y_test, pred))
    
    #d = {'sepal length (cm)' : [4.6],
    #    'sepal width (cm)' : [3.4],
    #    'petal length (cm)' : [1.4],
    #    'petal width (cm)' : [0.3],
    #    'species' : 0}
    d = {'sepal width (cm)' : [0.2],
        'petal length (cm)' : [4.2],
        'petal width (cm)' : [1.4],
        'species' : 0}
    
    test_df = pd.DataFrame(data= d)
    
    #print(test_df)    
    pred2 = lr.predict(test_df)
    
    #print('Predicted Sepal Length (cm):', pred2[0])
    #print('Actual Sepal Length (cm):', 4.6)
    #print(pred2)
    
    return mean_absolute_error(y_test, pred)
    

In [163]:
exploring_data = comp.create_component_from_func(exploring_dataset,output_component_file='iris/exploring_component.yaml',base_image='tensorflow/tensorflow:1.12.0-py3')

In [164]:
@component(
    packages_to_install=['scikit-learn'],
    base_image='python:3.9',
)
def iris_sgdclassifier(test_samples_fraction: float, metrics: Output[ClassificationMetrics]):
    from sklearn import datasets, model_selection
    from sklearn.linear_model import SGDClassifier
    from sklearn.metrics import confusion_matrix

    iris_dataset = datasets.load_iris()
    train_x, test_x, train_y, test_y = model_selection.train_test_split(
        iris_dataset['data'], iris_dataset['target'], test_size=test_samples_fraction)


    classifier = SGDClassifier()
    classifier.fit(train_x, train_y)
    predictions = model_selection.cross_val_predict(classifier, train_x, train_y, cv=3)
    metrics.log_confusion_matrix(
        ['Setosa', 'Versicolour', 'Virginica'],
        confusion_matrix(train_y, predictions).tolist() # .tolist() to convert np array to list.
    )

In [165]:
@component
def markdown_visualization(markdown_artifact: Output[Markdown]):
    markdown_content = '## Hello world \n\n Markdown content'
    with open(markdown_artifact.path, 'w') as f:
        f.write(markdown_content)

In [168]:
#def pipeline_iris():
#    exploring = exploring_data()
@dsl.pipeline(name='metrics-visualization-pipeline')
def metrics_visualization_pipeline():
    #iris_sgdclassifier_op = iris_sgdclassifier(test_samples_fraction=0.4)
    md = markdown_visualization()
    #e = exploring_data(0.50)
    

In [170]:
client = kfp.Client(host='http://localhost:8080/pipeline')
FILE = ''
args = {}
##print(client.list_experiments())
#client.create_run_from_pipeline_func(metrics_visualization_pipeline,arguments=args)
client.create_run_from_pipeline_func(metrics_visualization_pipeline,mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE,arguments=args)

RunPipelineResult(run_id=fcd34fbe-0dd3-4974-b304-b4cc9e88ce5a)